In [1]:
import pandas as pd
import numpy as np

In [23]:
data = pd.read_csv('G:\Github\Recipe_reco\data\RAW_recipes.csv')

In [27]:
data.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8


In [25]:
convert_to_list = ['tags','nutrition','steps','ingredients']

for c in convert_to_list:
    data[c] = data[c].apply(lambda x: eval(x))

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
name              231636 non-null object
id                231637 non-null int64
minutes           231637 non-null int64
contributor_id    231637 non-null int64
submitted         231637 non-null object
tags              231637 non-null object
nutrition         231637 non-null object
n_steps           231637 non-null int64
steps             231637 non-null object
description       226658 non-null object
ingredients       231637 non-null object
n_ingredients     231637 non-null int64
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [29]:
data.to_pickle('data/raw_df.pkl')

In [30]:
ingredients_all = data.ingredients.apply(lambda x: ", ".join(x))

0         winter squash, mexican seasoning, mixed spice,...
1         prepared pizza crust, sausage patty, eggs, mil...
2         ground beef, yellow onions, diced tomatoes, to...
3         spreadable cheese with garlic and herbs, new p...
4         tomato juice, apple cider vinegar, sugar, salt...
                                ...                        
231632    celery, onion, green sweet pepper, garlic clov...
231633    paprika, salt, garlic powder, onion powder, dr...
231634    hard-cooked eggs, mayonnaise, dijon mustard, s...
231635    butter, eagle brand condensed milk, light brow...
231636    granulated sugar, shortening, eggs, flour, cre...
Name: ingredients, Length: 231637, dtype: object

In [33]:
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
ingredients_all = data.ingredients.apply(lambda x: ", ".join(x))

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b')

# Fit and transform
X = vect.fit_transform(ingredients_all)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [34]:
#fit LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus,num_topics = 10,passes = 25, random_state = 34, id2word = id_map)

In [35]:
ldamodel.print_topics(num_topics = 10)

[(0,
  '0.068*"pepper" + 0.053*"salt" + 0.039*"onion" + 0.037*"dried" + 0.037*"garlic" + 0.030*"oil" + 0.027*"fresh" + 0.022*"olive" + 0.021*"ground" + 0.020*"black"'),
 (1,
  '0.063*"water" + 0.053*"whole" + 0.052*"wheat" + 0.044*"yogurt" + 0.036*"salt" + 0.035*"plain" + 0.034*"yeast" + 0.032*"seeds" + 0.031*"apple" + 0.030*"flour"'),
 (2,
  '0.063*"sauce" + 0.053*"powder" + 0.051*"pepper" + 0.050*"ground" + 0.043*"garlic" + 0.035*"onion" + 0.033*"chili" + 0.031*"salt" + 0.025*"sugar" + 0.024*"cumin"'),
 (3,
  '0.072*"fresh" + 0.058*"pepper" + 0.056*"oil" + 0.049*"olive" + 0.044*"salt" + 0.034*"red" + 0.033*"garlic" + 0.027*"juice" + 0.024*"black" + 0.022*"lemon"'),
 (4,
  '0.080*"sugar" + 0.080*"flour" + 0.061*"baking" + 0.054*"salt" + 0.046*"butter" + 0.041*"powder" + 0.034*"eggs" + 0.032*"all" + 0.032*"purpose" + 0.032*"vanilla"'),
 (5,
  '0.130*"juice" + 0.084*"lemon" + 0.061*"orange" + 0.046*"sugar" + 0.042*"fresh" + 0.032*"cream" + 0.030*"half" + 0.029*"water" + 0.027*"lime" + 0